In [1]:
#Task 2 Part 1
#Contributors : Eunice Ofori Addo, Ajinkya Borle
"""We couldn't get a n_hin (computationally heavy + not get the correct expectation)
BUT  by using this code (essentially binary search) and maybe tweaking the RBM parameters 
it will get you the minimum n_hin in O(logN) time (N being the upper limit of the range we search in ) """
import numpy as np
import torch
from RBM_helper import RBM

import Rydberg_energy_calculator

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))

In [2]:
n_vis = training_data.shape[1]


In [5]:
#Let us try to figure out what n_hin can be
#Define all parameters for the RBM that won't change within the loop
epochs = 500
num_samples = 2000 #2000 samples to approximate the expectation
k = 100 #Having too huge of a k is painful, and some studies have shown k=1 to also work well

#Now we define the variables that are specific to our 'intelligent guesswork', inspired by binary search: https://en.wikipedia.org/wiki/Binary_search_algorithm
upper_n_hin = 500 - 1  #This variable is how high can we go for n_hin (0 based index that's why -1)
lower_n_hin = 0 #This variable is how low can we go for n_hin (0 based index that's why 0 and not 1)


temp_n_hin_cand = -1 #This variable holds the temporary candidate for final n_hin


exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)

while lower_n_hin <= upper_n_hin:
    
    cur_n_hin = int((upper_n_hin + lower_n_hin)/2) 
    print("Upper limit is (0 based):",upper_n_hin)
    print("Lower limit is: (0 based)",lower_n_hin)
    print("Current n_hin is:",cur_n_hin+1)
    rbm = RBM(n_vis, cur_n_hin + 1) #Create an rbm object here, +1 to offset the 0-based index
    
    for e in range(1, epochs+1):
        # do one epoch of training
        rbm.train(training_data)   

        # now generate samples and calculate the energy
        if e % 100 == 0:
            print("\nEpoch: ", e)
            print("Sampling...")

            init_state = torch.zeros(num_samples, n_vis)
            RBM_samples = rbm.draw_samples(k, init_state)

            print("Done sampling. Calculating energy...") 
            
            energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction) 
            print("Energy from RBM samples: ", energies.item())
    #After total epochs done
    print("---")
    if abs(energies.item() - exact_energy) < 0.0001:
        print("Criteria reached, let's see if we can get a better estimate of n_hin")
        temp_n_hin_cand = cur_n_hin + 1 #temp_n_hin_cand should capture the true value
        upper_n_hin = cur_n_hin - 1
    else:
        lower_n_hin = cur_n_hin + 1
        print("Criteria not reached")
if temp_n_hin_cand != -1:
    print("Best n_hin we found:", temp_n_hin_cand)
else:
    print("No suitable n_hin within this range")

Exact energy:  -4.1203519096
Upper limit is (0 based): 499
Lower limit is: (0 based) 0
Current n_hin is: 250

Epoch:  100
Sampling...
Done sampling. Calculating energy...


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Energy from RBM samples:  -4.1204057916962515

Epoch:  200
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.1198517410036395

Epoch:  300
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.11984900479955

Epoch:  400
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.119869526948391

Epoch:  500
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.119865044268914
---
Criteria not reached
Upper limit is (0 based): 499
Lower limit is: (0 based) 250
Current n_hin is: 375

Epoch:  100
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.11970070248269

Epoch:  200
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.119417059420856

Epoch:  300
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.120104917343137

Epoch:  400
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.12011776

KeyboardInterrupt: 